In [1]:
import tensorflow as tf

# 导入keras模型
from tensorflow import keras
# 导入keras的组件
from tensorflow.keras import layers, Sequential, optimizers, losses, metrics, datasets

In [2]:
# 加载本章需要的数据
def preprocess(x, y):
    x = tf.cast(x, dtype=tf.float32) / 255.
    x = tf.reshape(x, [28*28])
    y = tf.cast(y, dtype=tf.int32)
    y = tf.one_hot(y, depth=10)
    return x,y

batchsz = 128
(x, y), (x_val, y_val) = datasets.mnist.load_data()
print('datasets:', x.shape, y.shape, x.min(), x.max())

db = tf.data.Dataset.from_tensor_slices((x,y))
db = db.map(preprocess).shuffle(60000).batch(batchsz)
ds_val = tf.data.Dataset.from_tensor_slices((x_val, y_val))
ds_val = ds_val.map(preprocess).batch(batchsz) 

sample = next(iter(db))

datasets: (60000, 28, 28) (60000,) 0 255


# 08. Keras高层接口
`TensorFlow`与`Keras`之间存在既竞争又合作的交错关系，甚至`Keras`创始人都在`Google`工作。早在2015年11月，`TensorFlow`就被加入`Keras`后端支持。从2017年开始，`Keras`的大部分组件被整合到`TensorFlow`框架中。2019年，在`TensorFlow 2`版本中，`Keras`被正式确定为`TensorFlow`的高层唯一接口API，取代了`TensorFlow 1`版本中自带的`tf.layers`等高层接口。也就是说，现在只能使用`Keras`的接口来完成`TensorFlow`层方式的模型搭建与训练。在`TensorFlow`中，`Keras`被实现在`tf.keras`子模块中。

`Keras`与`tf.keras`有什么区别与联系呢？其实`Keras`可以理解为一套搭建与训练神经网络的高层API协议，`Keras`本身已经实现了此协议，安装标准的`Keras`库就可以方便地调用`TensorFlow`、`CNTK`等后端完成加速计算；在`TensorFlow`中，也实现了一套`Keras`协议，即`tf.keras`，它与`TensorFlow`深度融合，且只能基于`TensorFlow`后端运算，并对`TensorFlow`的支持更完美。对于使用`TensorFlow`的开发者来说，`tf.keras`可以理解为一个普通的子模块，与其他子模块，如`tf.math`，`tf.data`等并没有什么差别。下文如无特别说明，`Keras`均指代`tf.keras`，而不是标准的`Keras`库。

## 8.1 常见功能模块

### 8.1.1 常见网络层类
在`tf.keras.layers`命名空间(下文使用`layers`指代`tf.keras.layers`)中提供了大量常见网络层的类，如`全连接层`、`激活函数层`、`池化层`、`卷积层`、`循环神经网络层`等。

以`Softmax`层为例演示一下：

In [3]:
x = tf.constant([2.,1.,0.1])
# 创建Softmax层
layer = layers.Softmax(axis=-1)
# 调用softmax前向计算，输出为out
out = layer(x)
out

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.6590012 , 0.24243298, 0.09856589], dtype=float32)>

当然，也可以直接通过`tf.nn.softmax()`函数完成计算，代码如下：

In [4]:
out = tf.nn.softmax(x)
out

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.6590012 , 0.24243298, 0.09856589], dtype=float32)>

### 8.1.2 网络容器
可以通过`Keras`提供的网络容器`Sequential`将多个网络层封装成一个大网络模型，只需要调用网络模型的实例一次即可完成数据从第一层到最末层的顺序传播运算。

In [5]:
network = Sequential([
    layers.Dense(3, activation=None), # 全连接层，此处不使用激活函数
    layers.ReLU(),                    # 激活函数层
    layers.Dense(2, activation=None), # 全连接层，此处不使用激活函数
    layers.ReLU()                     # 激活函数层
])

x = tf.random.normal([4,3])
out = network(x)
network.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  12        
_________________________________________________________________
re_lu (ReLU)                 multiple                  0         
_________________________________________________________________
dense_1 (Dense)              multiple                  8         
_________________________________________________________________
re_lu_1 (ReLU)               multiple                  0         
Total params: 20
Trainable params: 20
Non-trainable params: 0
_________________________________________________________________


`Sequential`容器也可以通过`add()`方法继续追加新的网络层，实现动态创建网络的功能：

In [6]:
layers_num = 2           # 堆叠2次
network = Sequential([]) # 先创建空的网络容器

for _ in range(layers_num):
    network.add(layers.Dense(3))      # 添加全连接层
    network.add(layers.ReLU())        # 添加激活函数层
    network.build(input_shape=(4, 4)) # 创建网络参数

通过指定任意的`layers_num`参数即可创建对应层数的网络结构，在完成网络创建时，网络层类并没有创建内部权值张量等成员变量，此时通过调用类的`build`方法并指定输入大小，即可自动创建所有层的内部张量。

通过`summary()`函数可以方便打印出网络结构和参数量，打印结果如下：

In [7]:
network.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              multiple                  15        
_________________________________________________________________
re_lu_2 (ReLU)               multiple                  0         
_________________________________________________________________
dense_3 (Dense)              multiple                  12        
_________________________________________________________________
re_lu_3 (ReLU)               multiple                  0         
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________


可以看到`Layer`列为每层的名字，这个名字由`TensorFlow`内部维护。`Param#`列为层的参数个数，`Total params`项统计出了总的参数量，`Trainable params`为总的待优化参数量，`Non-trainable params`为总的不需要优化的参数量。

当我们通过`Sequential`容量封装多个网络层时，每层的参数列表将会自动并入`Sequential`容器的参数列表中，不需要人为合并网络参数列表，这也是`Sequential`容器的便捷之处。`Sequential`对象的`trainable_variables`和`variables`包含了所有层的待优化张量列表和全部张量列表，例如：

In [8]:
# 打印网络的待优化参数名与shape
for p in network.trainable_variables:
    print(p.name, p.shape) # 参数名和形状

dense_2/kernel:0 (4, 3)
dense_2/bias:0 (3,)
dense_3/kernel:0 (3, 3)
dense_3/bias:0 (3,)


`Sequential`容器是最常用的类之一，对于快速搭建多层神经网络非常有用，应尽量多使用来简化网络模型的实现。

## 8.2 模型装配、训练与测试

### 8.2.1 模型装配
在`Keras`中，有2个比较特殊的类：
+ **keras.layers.Layer**：网络层的母类，定义了网络层的一些常见功能，如添加权值、管理权值列表等
+ **keras.Model**：网络的母类，除了具有`Layer`类的功能，还添加了保存模型、加载模型、训练与测试模型等便捷功能

`Sequential`也是`Model`的子类，因此具有`Model`类的所有功能。

接下来介绍`Model`及其子类的模型装配与训练功能：

In [9]:
# 创建5层的全连接网络
network = Sequential([
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(10)])

network.build(input_shape=(4, 28*28))
network.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              multiple                  200960    
_________________________________________________________________
dense_5 (Dense)              multiple                  32896     
_________________________________________________________________
dense_6 (Dense)              multiple                  8256      
_________________________________________________________________
dense_7 (Dense)              multiple                  2080      
_________________________________________________________________
dense_8 (Dense)              multiple                  330       
Total params: 244,522
Trainable params: 244,522
Non-trainable params: 0
_________________________________________________________________


创建网络后，正常的流程是循环迭代数据集多个`Epoch`，每次按批产生训练数据、前向计算，然后通过损失函数计算误差值，并反向传播自动计算梯度、更新网络参数。这一部分逻辑由于非常通用，在`Keras`中提供了`compile()`和`fit()`函数方便实现上述逻辑。首先通过`compile`函数指定网络使用的优化器对象、损失函数类型，评价指标等设定，这一步称为`装配`。例如：

In [10]:
# 采用Adam优化器，学习率为0.01; 采用交叉熵损失函数，包含Softmax
network.compile(
    optimizer=optimizers.Adam(lr=0.01),
    loss=losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'] # 设置测量指标为准确率
)

### 8.2.2 模型训练
模型装配完成后，即可通过`fit()`函数送入待训练的数据集和验证用的数据集，这一步称为模型训练。

In [11]:
# 指定训练集为db, 验证集为ds_val, 训练5个epoch, 每2个epoch验证一次
history = network.fit(db, epochs=5, validation_data=ds_val, validation_freq=2)

Train for 469 steps, validate for 79 steps
Epoch 1/5
469/469 [==============================] - 5s 12ms/step - loss: 0.2611 - accuracy: 0.9212
Epoch 2/5
469/469 [==============================] - 6s 13ms/step - loss: 0.1305 - accuracy: 0.9628 - val_loss: 0.1218 - val_accuracy: 0.9666
Epoch 3/5
469/469 [==============================] - 4s 9ms/step - loss: 0.1061 - accuracy: 0.9698
Epoch 4/5
469/469 [==============================] - 5s 12ms/step - loss: 0.0944 - accuracy: 0.9738 - val_loss: 0.1261 - val_accuracy: 0.9699
Epoch 5/5
469/469 [==============================] - 5s 10ms/step - loss: 0.0906 - accuracy: 0.9755


In [12]:
history.history

{'loss': [0.26117627529303233,
  0.13056919217904409,
  0.1060667632261912,
  0.09437981604536375,
  0.09056901097098986],
 'accuracy': [0.9212, 0.96283334, 0.9697667, 0.9737833, 0.9755167],
 'val_loss': [0.12180394387385467, 0.12608745675813995],
 'val_accuracy': [0.9666, 0.9699]}

### 8.2.3 模型测试
`Model`类除了可以完成网络的装配与训练、验证，还可以非常方便的预测和测试：
```python
# 加载一个 batch 的测试数据
x, y = next(iter(db_test))
print('predict x:', x.shape) # 打印当前 batch 的形状

out = network.predict(x) # 模型预测，预测结果保存在 out 中
print(out)
```

其中`out`即为网络的输出。通过上述代码即可使用训练好的模型去预测新样本的标签信息。

如果只是简单的测试模型的性能，可以通过`Model.evaluate(db)`循环测试完`db`数据集上所有样本，并打印出性能指标，例如：
```python
network.evaluate(db_test) # 模型测试，测试在db_test上的性能表现
```

## 8.3 模型保存与加载
模型训练完成后可以保存到文件系统上。实际上，在训练时间隔性地保存模型状态也是非常好的习惯。一般大规模的网络训练时间非常长，一旦发生宕机等意外，之前训练的进度将全部丢失。如果能够间断地保存模型，那么，发生宕机时可以从最近一次的网络状态文件中恢复。

`Keras`有三种常用的模型保存与加载方法。

### 8.3.1 张量方式
网络的状态主要体现在网络的结构以及网络层内部张量数据上，因此在拥有网络结构源文件的条件下，直接保存网络张量参数到文件系统上是最轻量级的一种方式。

In [13]:
network.save_weights('./models/weights.ckpt') # 保存模型的所有张量数据

在需要的时候，先创建好网络对象，然后调用网络对象的`load_weights(path)`方法即可将指定的模型文件中保存的张量数值写入到当前网络参数中去。

In [14]:
del network # 删除网络对象

# 重新创建相同的网络结构
network = Sequential([
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(10)])

network.compile(
    optimizer=optimizers.Adam(lr=0.01),
    loss=tf.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# 从参数文件中读取数据并写入当前网络
network.load_weights('./models/weights.ckpt')

这种保存与加载网络的方式最为轻量级，文件中保存的仅仅是张量参数的数值，并没有其它额外的结构参数。但是它需要使用相同的网络结构才能够正确恢复网络状态，因此一般在拥有网络源文件的情况下使用。

### 8.3.2 网络方式
通过`Model.save(path)`函数可以将模型的结构以及模型的参数保存到文件上，在不需要网络源文件的条件下，通过`keras.models.load_model(path)`即可恢复网络结构和网络参数。

```python
# 保存模型结构与模型参数到文件
network.save('./models/model.h5')

# 删除网络对象
del network

# 从文件恢复网络结构与网络参数
network = keras.models.load_model('./models/model.h5')
```

`model.h5`文件除了保存了模型参数外，还应保存了网络结构信息，不需要提前创建模型即可直接从文件中恢复出网络`network`对象。

### 8.3.3 SavedModel方式
`TensorFlow`之所以能够被业界青睐，除了优秀的神经网络层API支持之外，还得益于它强大的生态系统，包括移动端和网页端等的支持。当需要将模型部署到其他平台时，采用`TensorFlow`的`SavedModel`方式存储模型具有平台无关性。

```python
# 保存模型结构与模型参数到文件
tf.saved_model.save(network, './models/model-savedmodel')

del network # 删除网络对象
```

通过`tf.saved_model.load`函数即可恢复出模型对象：
```python
# 从文件恢复网络结构与网络参数
network = tf.saved_model.load('model-savedmodel')
```

## 8.4 自定义网络
创建自定义网络层类时，需要继承自`layers.Layer`基类；创建自定义的网络类时，需要继承自`keras.Model`基类，这样建立的自定义类才能够方便的利用`Layer/Model`基类提供的参数管理等功能，同时也能够与其他的标准网络层类交互使用。

### 8.4.1 自定义网络层
对于自定义的网络层，至少需要实现初始化`__init__`方法和前向传播逻辑`call`方法。

我们定义一个没有偏置向量的全连接层，即`bias`为0，同时固定激活函数为`ReLU`函数。

In [15]:
class MyDense(layers.Layer):
    # 自定义网络层
    def __init__(self, inp_dim, outp_dim):
        super(MyDense, self).__init__()
        # 创建权值张量并添加到类管理列表中，设置为需要优化
        self.kernel = self.add_variable('w', [inp_dim, outp_dim], trainable=True)

需要注意的是，`self.add_variable`会返回张量$W$的`Python`引用，而变量名`name`由`TensorFlow`内部维护，使用的比较少。我们实例化`MyDense`类，并查看其参数列表：

In [16]:
# 创建输入为4，输出为3节点的自定义层
net = MyDense(4,3) 
# 查看自定义层的参数列表
net.variables, net.trainable_variables

Instructions for updating:
Please use `layer.add_weight` method instead.


([<tf.Variable 'w:0' shape=(4, 3) dtype=float32, numpy=
  array([[-0.40342063, -0.5115143 ,  0.19999051],
         [ 0.7019731 ,  0.34955537,  0.38899946],
         [ 0.49433422, -0.45593867, -0.28776813],
         [-0.0899775 , -0.1263079 ,  0.597908  ]], dtype=float32)>],
 [<tf.Variable 'w:0' shape=(4, 3) dtype=float32, numpy=
  array([[-0.40342063, -0.5115143 ,  0.19999051],
         [ 0.7019731 ,  0.34955537,  0.38899946],
         [ 0.49433422, -0.45593867, -0.28776813],
         [-0.0899775 , -0.1263079 ,  0.597908  ]], dtype=float32)>])

完成自定义类的初始化工作后，我们来设计自定义类的前向运算逻辑，对于这个例子，只需要完成$O = X@X$矩阵运算，并通过固定的$ReLU$激活函数即可，代码如下:

In [17]:
class MyDense(layers.Layer):
    # 自定义网络层
    def __init__(self, inp_dim, outp_dim):
        super(MyDense, self).__init__()
        # 创建权值张量并添加到类管理列表中，设置为需要优化
        self.kernel = self.add_variable('w', [inp_dim, outp_dim], trainable=True)
    
    # 实现自定义类的前向计算逻辑
    def call(self, inputs, training=None):
        # X@W
        out = inputs @ self.kernel
        # 执行激活函数运算
        out = tf.nn.relu(out)
        return out

前向运算逻辑实现在`call(inputs, training=None)`函数中，其中`inputs`代表输入，由用户在调用时传入；`training`参数用于指定模型的状态`training`为`True`时执行训练模式，`training`为`False`时执行测试模式。由于全连接层的训练模式和测试模式逻辑一致，此处不需要额外处理。对于部份测试模式和训练模式不一致的网络层，需要根据`training`参数来设计需要执行的逻辑。

### 8.4.2 自定义网络
我们基于上述的`无偏置的全连接层`来实现`MNIST`手写数字图片模型的创建：

In [18]:
network = Sequential([
    MyDense(784, 256), 
    MyDense(256, 128),
    MyDense(128, 64),
    MyDense(64, 32),
    MyDense(32, 10)])

network.build(input_shape=(None, 28*28))
network.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
my_dense_1 (MyDense)         multiple                  200704    
_________________________________________________________________
my_dense_2 (MyDense)         multiple                  32768     
_________________________________________________________________
my_dense_3 (MyDense)         multiple                  8192      
_________________________________________________________________
my_dense_4 (MyDense)         multiple                  2048      
_________________________________________________________________
my_dense_5 (MyDense)         multiple                  320       
Total params: 244,032
Trainable params: 244,032
Non-trainable params: 0
_________________________________________________________________


`Sequential`容器适合于数据按序从第一层传播到第二层，再从第二层传播到第三层，以此规律传播的网络模型。对于复杂的网络结构，例如第三层的输入不仅是第二层的输出，还有第一层的输出，此时使用自定义网络更加灵活。自定义网络类继承自`Model`基类：

In [19]:
class MyModel(keras.Model):
    # 自定义网络类，继承自Model基类
    def __init__(self):
        super(MyModel, self).__init__()
        # 完成网络内需要的网络层的创建工作
        self.fc1 = MyDense(28*28, 256)
        self.fc2 = MyDense(256, 128)
        self.fc3 = MyDense(128, 64)
        self.fc4 = MyDense(64, 32)
        self.fc5 = MyDense(32, 10)
    
    # 自定义前向运算逻辑
    def call(self, inputs, training=None):
        x = self.fc1(inputs)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        x = self.fc5(x)
        return x

## 8.5 模型乐园
对于常用的网络模型，如`ResNet`、`VGG`等，不需要手动创建网络，可以直接从`keras.applications`子模块中通过一行代码即可创建并使用这些经典模型。

### 8.5.1 加载模型
以`ResNet50`网络模型为例，一般将`ResNet50`去除最后一层后的网络作为新任务的特征提取子网络，即利用在`ImageNet`数据集上预训练好的网络参数初始化，并根据自定义任务的类别追加一个对应数据类别数的全连接分类层或子网络，从而可以在预训练网络的基础上快速、高效地学习新任务。

In [20]:
# 加载ImageNet预训练网络模型，并去掉最后一层
resnet = keras.applications.ResNet50(weights='imagenet',include_top=False)
# resnet.summary()

In [21]:
# 测试网络的输出
x = tf.random.normal([4,224,224,3])
out = resnet(x) # 获得子网络的输出
out.shape

TensorShape([4, 7, 7, 2048])

上述代码自动从服务器下载模型结构和在`ImageNet`数据集上预训练好的网络参数。通过设置`include_top`参数为`False`，可以选择去掉`ResNet50`最后一层，此时网络的输出特征图大小为`[b,7,7,2048]`。对于某个具体的任务，需要设置自定义的输出节点数。我们可以新建一个池化层，将特征从`[b,7,7,2048]`降维到`[b,2048]`：

In [22]:
# 新建池化层
global_average_layer = layers.GlobalAveragePooling2D()
# 利用上一层的输出作为本层的输入，测试其输出
x = tf.random.normal([4,7,7,2048])
# 池化层降维，形状由[4,7,7,2048]变为[4,1,1,2048],删减维度后变为[4,2048]
out = global_average_layer(x)
out.shape

TensorShape([4, 2048])

最后新建一个全连接层，并设置输出节点数为100：

In [23]:
# 新建全连接层
fc = layers.Dense(100)
# 利用上一层的输出[4,2048]作为本层的输入，测试其输出
x = tf.random.normal([4,2048])
out = fc(x)
out.shape

TensorShape([4, 100])

在创建预训练的`ResNet50`特征子网络、新建的池化层和全连接层后，利用`Sequential`容器封装成一个新的网络：

In [24]:
# 重新包裹成我们的网络模型
mynet = Sequential([resnet, global_average_layer, fc])
mynet.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, None, None, 2048)  23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense_14 (Dense)             (None, 100)               204900    
Total params: 23,792,612
Trainable params: 23,739,492
Non-trainable params: 53,120
_________________________________________________________________


通过设置`resnet.trainable = False`可以选择冻结`ResNet`部分的网络参数，只训练新建的网络层，从而快速、高效完成网络模型的训练。当然也可以在自定义任务上更新网络的全部参数。

## 8.6 测量工具
在网络的训练过程中，经常需要统计准确率、召回率等测量指标，除了可以通过手动计算的方式获取这些统计数据外，`Keras`提供了一些常用的测量工具，位于`keras.metrics`模块中，专门用于统计训练过程中常用的指标数据。

`Keras`的测量工具的使用方法一般有4个主要步骤：
1. 新建测量器
2. 写入数据
3. 读取统计数据
4. 清零测量器

### 8.6.1 新建测量器
`keras.metrics`模块提供了很多常用测量器类，如统计平均值的`Mean`类，统计准确率的`Accuracy`类，统计余弦相似度的`CosineSimilarity`类等。

下面我们以统计误差 值为例。在前向运算时，我们会得到每一个`Batch`的平均误差，但是我们希望统计每个`Step`的平均误差，因此选择使用`Mean`测量器：

```python
# 新建平均测量器，适合Loss数据
loss_meter = metrics.Mean()
```

### 8.6.2 写入数据
通过测量器的`update_state`函数可以写入新的数据，测量器会根据自身逻辑记录并处理采样数据。例如，在每个`Step`结束时采集一次`loss`值，代码如下：

```python
# 记录采样的数据，通过float()函数将张量转换为普通数值
loss_meter.update_state(float(loss))
```

上述采样代码放置在每个`Batch`运算结束后，测量器会自动根据采样的数据来统计平均值。

### 8.6.3 读取统计信息
在采样多次数据后，可以选择在需要的地方调用测量器的`result()`函数，来获取统计值。

例如，间隔性统计`loss`均值，代码如下：

```python
# 打印统计期间的平均 loss
print(step, 'loss:', loss_meter.result())
```

### 8.6.4 清除状态
由于测量器会统计所有历史记录的数据，因此在启动新一轮统计时，有必要清除历史状态。通过`reset_states()`：

```python
if step % 100 == 0:
    # 打印统计的平均 loss
    print(step, 'loss:', loss_meter.result())
    loss_meter.reset_states() # 打印完后，清零测量器
```

### 8.6.5 准确率统计实战
按照测量工具的使用方法，我们利用准确率测量器`Accuracy`类来统计训练过程中的准确率。

首先新建准确率测量器：

```python
acc_meter = metrics.Accuracy() # 创建准确率测量器
```

在每次前向计算完成后，记录训练准确率数据。需要注意的是，`Accuracy`类的`update_state`函数的参数为预测值和真实值，而不是当前`Batch`的准确率：

```python
# [b, 784] => [b, 10]，网络输出值
out = network(x)
# [b, 10] => [b]，经过 argmax 后计算预测值
pred = tf.argmax(out, axis=1)
pred = tf.cast(pred, dtype=tf.int32)
# 根据预测值与真实值写入测量器
acc_meter.update_state(y, pred)
```

在统计完测试集所有`Batch`的预测值后，打印统计的平均准确率，并清零测量器：

```python
# 读取统计结果
print(step, 'Evaluate Acc:', acc_meter.result().numpy())
acc_meter.reset_states() # 清零测量器
```

## 8.7 可视化
+ TensorBoard
+ Visdom